In [78]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import plotly.offline as pl_off
pl_off.init_notebook_mode(connected=True)
import requests
import pandas as pd

In [79]:
url_parameters = dict(
    base_url = "pokeapi.co",
    directory = "api/v2/pokemon",
    pokemon_id = ''
)

endpoint  = "http://{base_url}/{directory}/{pokemon_id}".format(**url_parameters)

# Getting a result from the api that includes the number of pokemon
first_request = requests.get(endpoint)
first_results = first_request.json()

# Getting a result containing all of the pokemon's names and api urls
second_request = requests.get(endpoint + '?offset=0&limit='+str(first_results['count']))
second_results = second_request.json()

# Storing the Pokemon names and urls for later use
name_url = dict()
for i in second_results['results']:
    name_url[i['name'].capitalize()]=i['url']

pokemon_dataframe = pd.DataFrame()


In [77]:
name_url

{'Bulbasaur': 'https://pokeapi.co/api/v2/pokemon/1/',
 'Ivysaur': 'https://pokeapi.co/api/v2/pokemon/2/',
 'Venusaur': 'https://pokeapi.co/api/v2/pokemon/3/',
 'Charmander': 'https://pokeapi.co/api/v2/pokemon/4/',
 'Charmeleon': 'https://pokeapi.co/api/v2/pokemon/5/',
 'Charizard': 'https://pokeapi.co/api/v2/pokemon/6/',
 'Squirtle': 'https://pokeapi.co/api/v2/pokemon/7/',
 'Wartortle': 'https://pokeapi.co/api/v2/pokemon/8/',
 'Blastoise': 'https://pokeapi.co/api/v2/pokemon/9/',
 'Caterpie': 'https://pokeapi.co/api/v2/pokemon/10/',
 'Metapod': 'https://pokeapi.co/api/v2/pokemon/11/',
 'Butterfree': 'https://pokeapi.co/api/v2/pokemon/12/',
 'Weedle': 'https://pokeapi.co/api/v2/pokemon/13/',
 'Kakuna': 'https://pokeapi.co/api/v2/pokemon/14/',
 'Beedrill': 'https://pokeapi.co/api/v2/pokemon/15/',
 'Pidgey': 'https://pokeapi.co/api/v2/pokemon/16/',
 'Pidgeotto': 'https://pokeapi.co/api/v2/pokemon/17/',
 'Pidgeot': 'https://pokeapi.co/api/v2/pokemon/18/',
 'Rattata': 'https://pokeapi.co/api

In [80]:
def update_dataframe(input_pokemon_choices):
    global pokemon_dataframe
    pokedex = dict()
    for i in input_pokemon_choices:
        # Adds a pokemon's data to the dataframe if it wasn't already added this session
        if pokemon_dataframe.empty or i not in pokemon_dataframe['Name'].values:
            temp_results = requests.get(name_url[i]).json()

            temp_poke_df = pd.DataFrame(temp_results['stats'])
            dat = [list(temp_poke_df['base_stat'])]
            dat[0].append(i.capitalize())
            cols = [x['name'].capitalize() for x in temp_poke_df['stat']]
            cols.append('Name')
            pokemon_dataframe = pokemon_dataframe.append(
                pd.DataFrame(
                    data=dat, columns=cols
                )
            )


In [81]:
pokemon_dataframe = pd.DataFrame()

In [83]:
update_dataframe(['Bulbasaur'])
pokemon_dataframe

,Speed,Special-defense,Special-attack,Defense,Attack,Hp,Name
0,45,65,65,49,49,45,Bulbasaur


In [89]:
# @app.callback(
#     dash.dependencies.Output('base_stat_bar_graph', 'figure'),
#     [dash.dependencies.Input('pokemon_choices','value')])
def update_graph_group_bar(input_pokemon_choices):
    if type(input_pokemon_choices)==str:
        input_pokemon_choices = [input_pokemon_choices]

    update_dataframe(input_pokemon_choices)

    current_df = pokemon_dataframe[pokemon_dataframe['Name'].isin(input_pokemon_choices)]
# List comprehension to create data for the graph
    traces = [
        go.Bar({
            'x':list(current_df['Name']),
            'y':list(current_df[col]),
            'name':col
        }) for col in current_df.columns if col != 'Name']
    

    layout = {
        'barmode':'group'
    }

    fig={'data':traces, 'layout':go.Layout(layout)}
    pl_off.iplot( go.Figure(fig))

In [116]:
# @app.callback(
#     dash.dependencies.Output('base_stat_bar_graph', 'figure'),
#     [dash.dependencies.Input('pokemon_choices','value')])
def update_graph_stacked_bar(input_pokemon_choices, mode):
    if type(input_pokemon_choices)==str:
        input_pokemon_choices = [input_pokemon_choices]

    update_dataframe(input_pokemon_choices)

    current_df = pokemon_dataframe[pokemon_dataframe['Name'].isin(input_pokemon_choices)]
# List comprehension to create data for the graph
#     traces = [
#         go.Bar({
#             'x':list(current_df['Name']),
#             'y':list(current_df[col]),
#             'name':col
#         }) for col in current_df.columns if col != 'Name']
    

#     layout = {
#         'barmode':mode
#     }

#     fig={'data':traces, 'layout':go.Layout(layout)}
    
    traces = [
        {
            'labels':[col for col in current_df.columns if col != 'Name'],
            'values':current_df[current_df['Name']==name].values[0][:-1],
            'name':name,
            'textinfo':'value',
            'hoverinfo':'label+value',
            'type':'pie',
            'title':name,
        } for name in current_df['Name'].values
    ]

    layout = {'title': 'Multiple Pie Charts',
              'showlegend': True}

    fig={'data':traces, 'layout':go.Layout(layout)}

    pl_off.iplot( go.Figure(fig))

In [128]:
import math
int(math.sqrt(6))

2

In [142]:
a = go.Table(
                        header=dict(values=list(pokemon_dataframe.columns),
                        fill = dict(color='#C2D4FF'),
                        align = ['left']),
                        cells=dict(
                            values=[pokemon_dataframe[x] for x in pokemon_dataframe.columns],
                            fill = dict(color='#F5F8FF'),
                            align = ['left'] * 7)
                    )
pl_off.iplot([a])

In [133]:
import math
input_pokemon_choices = ['Bulbasaur', 'Squirtle', 'Pikachu', 'Charmander']
current_df = pokemon_dataframe[pokemon_dataframe['Name'].isin(input_pokemon_choices)]
traces = []

row = 0
col = 0
num_row_col = math.sqrt(len(input_pokemon_choices))
if num_row_col%1!=0:
    num_row_col+=1
num_row_col=int(num_row_col)


for name in input_pokemon_choices:    
    traces.append({
        'labels':[col for col in current_df.columns if col != 'Name'],
        'values':current_df[current_df['Name']==name].values[0][:-1],
        'name':name,
        'textinfo':'value',
        'hoverinfo':'label+value',
        'type':'pie',
        'title':name,
        'domain':{'row':row,'column':col},
    })
    
    col+=1
    if col >= num_row_col:
        col = 0
        row += 1

if len(input_pokemon_choices) == 2:
    layout = {'title': 'Multiple Pie Charts',
          'showlegend': True,
          'grid':go.layout.Grid(rows=1,columns=num_row_col)}

else:
    layout = {'title': 'Multiple Pie Charts',
          'showlegend': True,
          'grid':go.layout.Grid(rows=num_row_col,columns=num_row_col)}


fig={'data':traces, 'layout':go.Layout(layout)}

pl_off.iplot( go.Figure(fig))

In [111]:
current_df = pokemon_dataframe[pokemon_dataframe['Name'].isin(['Bulbasaur'])]
current_df[current_df['Name']=='Bulbasaur'].values[0][:-1]
#current_df

array([45, 65, 65, 49, 49, 45], dtype=object)

In [92]:
choices = [
    'Bulbasaur',
    #'Pikachu',
    #'Charmander',
    'Squirtle',
]
update_graph_group_bar(choices)

In [ ]:
go.Pie()

In [117]:
#group stack relative overlay (opacity?)
update_graph_stacked_bar(choices, 'stack')